In [9]:
# Pandas y numpy para manipulación
import pandas as pd
import numpy as np
import glob as glob
import seaborn as sn
import os as os
# Librerias para gráficos 
import seaborn as sns
import matplotlib.pyplot as plt
# Herramientas de modelado
import graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

**Sobre este directorio**<br>

Cada archivo es de un administrador de red de un año específico.

Las columnas de cada archivo son:<br>

**● net_manager:**      código del administrador de la red regional<br>
**● purchase_area:**    código del área donde se compra la energía<br>
**● street:**           Nombre de la calle<br>
**● zipcode_fromy zipcode_to:** 2 columnas para el rango de códigos postales cubiertos, 4 números y 2 letras<br>
**● city:**             Nombre de la ciudad<br>
**● num_connections:**  Número de conexiones en el rango de códigos postales<br>
**● delivery_perc:**    porcentaje del consumo neto de electricidad o gas. Cuanto más bajo, más energía se devolvió a la red (por ejemplo, si tiene paneles solares)<br>
**● perc_of_active_connections:**   Porcentaje de conexiones activas en el rango de código postal<br>
**● type_of_connection:**           principal tipo de conexión en el rango de código postal. Para la electricidad son # fusibles X # amperios. Para gas es G4, G6, G10, G16, G25<br>
**● type_conn_perc:**   porcentaje de presencia del tipo principal de conexión en el rango del código postal<br>
**● annual_consume:**   Consumo anual. Kwh para electricidad, m3 para gas<br>
**● annual_consume_lowtarif_perc:** Porcentaje de consumo en horario de tarifa baja. De 22:00 a 07:00 y los fines de semana.<br>
**● smartmeter_perc:**  porcentaje de smartmeters en los rangos de código postal<br>

In [10]:
# merging the files

data1 = pd.read_csv('./dataset/Electricity/enexis_electricity/enexis_electricity_01012010.csv', encoding='latin1', dtype= str, sep=",")
data2 = pd.read_csv('./dataset/Electricity/enexis_electricity/enexis_electricity_01012011.csv', encoding='latin1', dtype= str, sep=",")
data3 = pd.read_csv('./dataset/Electricity/enexis_electricity/enexis_electricity_01012012.csv', encoding='latin1', dtype= str, sep=",")
data4 = pd.read_csv('./dataset/Electricity/enexis_electricity/enexis_electricity_01012013.csv', encoding='latin1', dtype= str, sep=",")
data5 = pd.read_csv('./dataset/Electricity/enexis_electricity/enexis_electricity_01012014.csv', encoding='latin1', dtype= str, sep=",")
data6 = pd.read_csv('./dataset/Electricity/enexis_electricity/enexis_electricity_01012015.csv', encoding='latin1', dtype= str, sep=",")
data7 = pd.read_csv('./dataset/Electricity/enexis_electricity/enexis_electricity_01012016.csv', encoding='latin1', dtype= str, sep=",")
data8 = pd.read_csv('./dataset/Electricity/enexis_electricity/enexis_electricity_01012017.csv', encoding='latin1', dtype= str, sep=",")
data9 = pd.read_csv('./dataset/Electricity/enexis_electricity/enexis_electricity_01012018.csv', encoding='latin1', dtype= str, sep=",")
data10 = pd.read_csv('./dataset/Electricity/enexis_electricity/enexis_electricity_01012019.csv', encoding='latin1', dtype= str, sep=",")
data11 = pd.read_csv('./dataset/Electricity/enexis_electricity/enexis_electricity_01012020.csv', encoding='latin1', dtype= str, sep=",")
# Mostramos los 5 primeros registros del df



In [11]:
# Evaluamos el porcentaje de datos nulos por columna
def describe_data(cols: list, data: pd.DataFrame) -> pd.DataFrame:
    
    diccionario = {
        'Null': data.shape[0] - data.count(), 
        '% Null': round((data.shape[0] - data.count()) / data.shape[0] * 100, 3),
        'Types': data.dtypes
    }    
    return pd.DataFrame(index=cols, data=diccionario)


In [57]:
#crear el dataframe con la columna del anio

data_anio=pd.DataFrame({'anio':[2011]})
df1=pd.concat([data1,data_anio],axis=1)
columna=2011
df1['anio'] = df1.apply(lambda x: 2011 if np.isnan(x['anio']) else x['anio'], axis=1)

<class 'pandas.core.frame.DataFrame'>


,anio
0,2011


In [59]:
#cambiar columna nan por anio en todas las filas
describe_data(df1.columns,df1)

,net_manager,purchase_area,street,zipcode_from,zipcode_to,city,delivery_perc,num_connections,perc_of_active_connections,type_conn_perc,type_of_connection,annual_consume,annual_consume_lowtarif_perc,smartmeter_perc,anio
0,Enexis B.V.,ENEXIS,Sasdijk,4251AB,4251AB,WERKENDAM,100,16,100,NaN,NaN,4282,25,0.0,2011.0
1,Enexis B.V.,ENEXIS,Sasdijk,4251AC,4251AC,WERKENDAM,100,11,100,NaN,NaN,5113,10,0.0,2011.0
2,Enexis B.V.,ENEXIS,Sasdijk,4251AD,4251AD,WERKENDAM,100,30,100,NaN,NaN,4809,34,0.0,2011.0
3,Enexis B.V.,ENEXIS,Nieuweweg,4251AE,4251AG,WERKENDAM,100,21,100,NaN,NaN,5015,44,0.0,2011.0
4,Enexis B.V.,ENEXIS,Koppenhof,4251AH,4251AH,WERKENDAM,100,12,100,NaN,NaN,3074,22,0.0,2011.0
5,Enexis B.V.,ENEXIS,Plein,4251AJ,4251AJ,WERKENDAM,100,26,100,NaN,NaN,5480,26,0.0,2011.0
6,Enexis B.V.,ENEXIS,Zevenhuizen,4251AK,4251AN,WERKENDAM,100,24,100,NaN,NaN,3191,34,0.0,2011.0
7,Enexis B.V.,ENEXIS,Zagerij,4251AP,4251AT,WERKENDAM,100,32,100,NaN,NaN,2664,44,0.0,2011.0
8,Enexis B.V.,ENEXIS,Sluisstraat,4251AV,4251AV,WERKENDAM,100,21,100,NaN,NaN,3244,49,0.0,2011.0
9,Enexis B.V.,ENEXIS,Oudsas,4251AW,4251AW,WERKENDAM,100,11,100,NaN,NaN,12125,19,0.0,2011.0


In [7]:
def agregar_anio(anio,df: pd.DataFrame)->pd.DataFrame:
    for x in range(1,data.shape[0],1):
        nueva_fila={f"anio":anio}
        data=data.append(nueva_fila,ignore_index=True)
        return data
data_new= agregar_anio(2010,data1)


UnboundLocalError: local variable 'data' referenced before assignment

In [ ]:
data_describe.loc[data_describe.index.isin(['country_name', 'state_name', 'place_name','place_with_parent_names']),:]

,type,Porcentaje de nulos,Nulos,Valores distintos,Ejemplos
place_name,object,0.0,23,1061,"[Mataderos, La Plata, Liniers, Centro, Gualegu..."
place_with_parent_names,object,0.0,0,1164,"[|Argentina|Capital Federal|Mataderos|, |Argen..."
country_name,object,0.0,0,1,[Argentina]
state_name,object,0.0,0,28,"[Capital Federal, Bs.As. G.B.A. Zona Sur, Buen..."
